# SQL Chat GPT
In this notebook, we will learn how to use a feature of langchain to turn Chat GPT into an agent knowledgeable about our database.

For Chat GPT, context is everything. Therefore, if GPT has our database as context and reference, we can ask it questions related to the data.

Langchain is a Python package with various useful integrations to use with Chat GPT. 

To access the data from our database, we need to provide GPT with a .db file containing the database information. We will use 

Import the necessary langchain libraries for the project.



In [6]:
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


ModuleNotFoundError: No module named 'langchain_community'

Store our OpenAI key as an environment variable and establish our connection to the OpenAI API.

In [6]:
import os

#os.environ['OPENAI_API_KEY'] = 'sk-proj-efe2vbgZEHT1ayhhoa8vT3BlbkFJGeOFBjmwbeCh4M4zoKgu'
client_gpt = OpenAI(temperature=0, verbose=True)

NameError: name 'OpenAI' is not defined

Load our database into a variable using the functions provided by langchain.

In [17]:
base_dados = SQLDatabase.from_uri("sqlite:///englishfootball.db")

Create the chain that connects our database to Chat GPT.

In [18]:
db_chain = SQLDatabaseChain.from_llm(client_gpt, base_dados, verbose=True)

From now on, we can use the `db_chain.run` method to communicate with our database.

In [19]:
db_chain.run("Who is the top scorer in the league?")



> Entering new SQLDatabaseChain chain...
Who is the top scorer in the league?
SQLQuery:SELECT appearances.team_id, teams.team_name, SUM(appearances.goals_for) AS total_goals
FROM appearances
INNER JOIN teams ON appearances.team_id = teams.team_id
GROUP BY appearances.team_id
ORDER BY total_goals DESC
LIMIT 1;
SQLResult: [('T-022', 'Manchester United', 8141)]
Answer:The top scorer in the league is Manchester United with 8141 goals.
> Finished chain.


'The top scorer in the league is Manchester United with 8141 goals.'

In [ ]:
db_chain.run("""Give me the 3 teams with the better defense """)



> Entering new SQLDatabaseChain chain...
Give me the 3 teams with the better defense 
SQLQuery:SELECT teams.team_name, standings.goals_against 
FROM teams 
INNER JOIN standings ON teams.team_id = standings.team_id 
ORDER BY standings.goals_against ASC 
LIMIT 3
SQLResult: [('Wigan Borough', 0), ('Aldershot', 0), ('Maidstone United', 0)]
Answer:Wigan Borough, Aldershot, Maidstone United
> Finished chain.


'Wigan Borough, Aldershot, Maidstone United'

In [ ]:
db_chain.run("""Why do Wigan Borough, Aldershot, and Maidstone United have the best defense?""")



> Entering new SQLDatabaseChain chain...
Why do Wigan Borough, Aldershot, and Maidstone United have the best defense?
SQLQuery:SELECT teams.team_name, standings.goals_against
FROM teams
INNER JOIN standings ON teams.team_id = standings.team_id
WHERE teams.team_name IN ('Wigan Borough', 'Aldershot', 'Maidstone United')
ORDER BY standings.goals_against ASC
LIMIT 5
SQLResult: [('Wigan Borough', 0), ('Aldershot', 0), ('Maidstone United', 0), ('Aldershot', 38), ('Wigan Borough', 39)]
Answer:Wigan Borough, Aldershot, and Maidstone United have the best defense because they have the lowest number of goals against in the standings table.
> Finished chain.


'Wigan Borough, Aldershot, and Maidstone United have the best defense because they have the lowest number of goals against in the standings table.'